# iminuit and an external minimizer

This is a topical tutorial. If you are new to iminuit, you should go through the basic tutorial first. 

It is possible to use another minimizer to find the minimum of a function, and then use iminuit to compute the error matrix. We will demonstrate this with a maximum-likelihood fit of a normal distribution, which is carried out with scipy.optimize.minimize. iminuit is then used to compute the parameter uncertainties.

In [1]:
from iminuit import Minuit
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:
# normally distributed data
x = norm(0, 1).rvs(1000)

# negative log-likelihood for a normal distribution
def nll(par):
    return -np.sum(norm(par[0], par[1]).logpdf(x))

# minimize nll with scipy.optimize.minimize
result = minimize(nll, np.ones(2))
result

      fun: 1420.4531864771172
 hess_inv: array([[1.01212047e-03, 6.89058301e-06],
       [6.89058301e-06, 5.03511850e-04]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 56
      nit: 11
     njev: 14
   status: 0
  success: True
        x: array([0.00465721, 1.00151579])

In [3]:
# initialize Minuit with the fit result from scipy.optimize.minimize
m = Minuit.from_array_func(nll, result.x, pedantic=False)
m.hesse() # this also works without calling MIGRAD before

------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x0   |   0.005   |   0.045   |            |            |         |         |       |
| 1 | x1   |   1.00    |   0.03    |            |            |         |         |       |
------------------------------------------------------------------------------------------

If the confidence intervals produced by HESSE are sufficient, iminuit can be used effectively together with other minimizers.

We can also compute the "Hesse errors" at any other point than the minimum. These cannot be interpreted as confidence intervals, they are just some numbers related to the second derivative of the cost function at that point.


In [4]:
m.args[:] = (1.0, 0.5)
m.hesse()

------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x0   |   1.00    |   0.04    |            |            |         |         |       |
| 1 | x1   |   0.500   |   0.008   |            |            |         |         |       |
------------------------------------------------------------------------------------------

Using MINOS does not work (yet...), however. MINOS requires an existing function minimum produced by MIGRAD, which contains more information than just the parameter values.

In [5]:
try:
    m.minos()
except:
    import traceback
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-5-068aa930b7fa>", line 2, in <module>
    m.minos()
  File "iminuit/_libiminuit.pyx", line 917, in iminuit._libiminuit.Minuit.minos
RuntimeError: MINOS require function to be at the minimum. Run MIGRAD first.
